In [ ]:
import pandas as pd
import pulp
from typing import List, Tuple, Dict


In [ ]:

import pandas as pd
import pulp
import matplotlib.pyplot as plt
import numpy as np
from typing import List, Tuple, Dict

plt.style.use("ggplot")

df = pd.read_csv('Nuclear waste management.csv', index_col=0)

preorder = {
    1: [5],
    2: [1],
    3: [3, 7],
    4: [11],
    5: [14],
    6: [22, 23],
    7: [24],
    8: [27]
}

preferential_info = [(5,1),
                     (1,3),
                     (3,11),
                     (11,14),
                     (14,22),
                     (22,24),
                     (24,27)]

indifference_information = [(3, 7),
                            (22, 23)]

In [ ]:

criteria = df.columns.tolist()
alternatives = {x for t in (preferential_info + indifference_information) for x in t}

pulp.LpSolverDefault.msg = 0
problem = pulp.LpProblem("UTA", pulp.LpMaximize)
print("Kryteria:", criteria)

# zmienne dla wartości funkcji użyteczności wariantów z rankingu
u_vars = {}
for alternative in alternatives:
    for criterion in criteria:
        value = df.loc[alternative, criterion]
        criterion_no = criteria.index(criterion) + 1
    
        u_vars[(criterion, value)] = pulp.LpVariable(
            f"u{criterion_no}({value})", lowBound=0, upBound=1
        )

print(u_vars)



In [ ]:
epsilon = pulp.LpVariable("epsilon", lowBound=0)
problem += epsilon

# preporządek
for a, b in preferential_info:
    problem += (
        pulp.lpSum(u_vars[(c, df.loc[a, c])] for c in criteria)
        >= pulp.lpSum(u_vars[(c, df.loc[b, c])] for c in criteria) + epsilon
    )

for a, b in indifference_information:
    problem += pulp.lpSum(u_vars[(c, df.loc[a, c])] for c in criteria) == pulp.lpSum(
        u_vars[(c, df.loc[b, c])] for c in criteria
    )

In [ ]:
worst_values = {criterion: df[criterion].max() for criterion in criteria}
best_values = {criterion: df[criterion].min() for criterion in criteria}

breakpoints = {criterion: sorted(df[criterion].unique()) for criterion in criteria}

u_best = []
u_worst = []


for criterion, value in worst_values.items():
    if (criterion, value) not in u_vars:
        criterion_no = criteria.index(criterion) + 1
        u_vars[(criterion, value)] = pulp.LpVariable(f"u{criterion_no}({value})", lowBound=0, upBound=1)

    u_worst.append(u_vars[(criterion, value)])

for criterion, value in best_values.items():
    if (criterion, value) not in u_vars:
        criterion_no = criteria.index(criterion) + 1
        u_vars[(criterion, value)] = pulp.LpVariable(f"u{criterion_no}({value})", lowBound=0, upBound=1)
        
    u_best.append(u_vars[(criterion, value)])


for criterion in breakpoints.keys():
    for value in breakpoints[criterion]:
        if (criterion, value) not in u_vars:
            criterion_no = criteria.index(criterion) + 1
            u_vars[(criterion, value)] = pulp.LpVariable(f"u{criterion_no}({value})", lowBound=0, upBound=1)


problem += pulp.lpSum(u_worst) == 0
problem += pulp.lpSum(u_best) == 1

weights = [1/len(criteria) for _ in criteria]

for criterion in criteria:
    problem += u_vars[(criterion, best_values[criterion])] == 1 * weights[criteria.index(criterion)]

    for i, value in enumerate(breakpoints[criterion]):

        if value == best_values[criterion]:
            continue

        key1 = (criterion, breakpoints[criterion][i])
        key2 = (criterion, breakpoints[criterion][i - 1])

        problem += u_vars[key1] <= u_vars[key2]   

In [ ]:
print(problem)

In [ ]:
problem.solve()
print("Status:", pulp.LpStatus[problem.status])

for v in problem.variables():
    print(v.name, "=", v.varValue)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))  # Create a figure and a 2x2 grid of subplots

axs = axs.flatten()  # Flatten the array of axes to make it easier to iterate over

for i, criterion in enumerate(criteria):
    x = breakpoints[criterion]
    print(x)
    y = [u_vars[(criterion, value)].varValue for value in x]
    axs[i].scatter(x, y)  # Plot on the i-th subplot
    axs[i].set_title(criterion)  # Set the title of the subplot to the criterion

plt.tight_layout()  # Adjust the layout to prevent overlap
plt.show()  # Display the figure